## Getting Started

In [83]:
# Code cell to be removed in the final version
from IPython.display import Javascript

keep_alive = Javascript('''
    function KeepAlive(){
        console.log("Keeping notebook alive");
        setInterval(function(){
            var xhr = new XMLHttpRequest();
            xhr.open("GET", "/", true);
            xhr.send();
        }, 60000);  // 60 seconds interval
    }
    KeepAlive();
''')

display(keep_alive)

<IPython.core.display.Javascript object>

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

## Gene Expression Data

In [85]:
# Load gene expression data
gene_expression_file = "dataset/GSE49711_SEQC_NB_TAV_G_log2.final.txt"
gene_data = pd.read_csv(gene_expression_file, sep='\t')

In [86]:
# veiw data
gene_data.head(10)

,Gene,Gene_set,NCBI_gene_ID,RefSeq_transcript_ID,Chromosome,Strand,Start,End,SEQC_NB001,SEQC_NB002,...,SEQC_NB489,SEQC_NB490,SEQC_NB491,SEQC_NB492,SEQC_NB493,SEQC_NB494,SEQC_NB495,SEQC_NB496,SEQC_NB497,SEQC_NB498
0,2-oxoacid_dh,Gene_AceView,.,.,11,-,111933358,111934981,16.1928,16.5853,...,16.8862,17.7853,16.3760,16.9434,17.3701,17.8362,17.8373,17.4535,17.3726,16.8897
1,A1BGAS,Gene_AceView,503538,NR_015380.1,19,+,58859074,58866555,17.2744,15.3742,...,16.8690,16.7470,15.6558,16.7318,16.6318,16.4946,16.8059,16.2498,17.6346,16.8966
2,A1CF,Gene_AceView,29974,NM_001198818.1;NM_001198819.1;NM_001198820.1;N...,10,-,52566307,52645436,8.7985,0.0000,...,0.0000,10.8696,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,11.4219,0.0000
3,A2BP1,Gene_AceView,54715,NM_001142333.1;NM_001142334.1;NM_018723.3;NM_1...,16,+,6068990,7763341,16.8905,18.6162,...,14.9643,16.0162,15.5955,15.4929,17.4806,20.0256,19.4085,19.4109,20.0640,19.3468
4,A2LD1,Gene_AceView,87769,NM_001195087.1;NM_033110.2,13,-,101182342,101241782,13.8812,14.0487,...,13.9839,16.1991,14.4934,15.0323,15.1812,14.7423,14.6655,13.7998,15.1968,14.5997
5,A2M,Gene_AceView,2,NM_000014.4,12,-,9220299,9280190,19.9015,19.9509,...,20.1619,21.5522,19.9990,20.8131,20.0009,21.6698,21.4414,20.6419,21.4051,22.2634
6,A2ML1,Gene_AceView,144568,NM_144670.4;NR_046256.1,12,+,8975069,9039597,13.5771,0.0000,...,11.2543,7.8841,0.0000,0.0000,0.0000,0.0000,11.5173,13.0990,11.6705,13.4238
7,A2MP1,Gene_AceView,3,NR_040112.1,12,-,9381129,9412153,11.7368,0.0000,...,11.4860,12.2344,0.0000,12.6326,10.0283,12.2452,11.5676,11.1459,0.0000,13.0892
8,A4GALT,Gene_AceView,53947,NM_017436.4,22,-,43088118,43117306,15.1902,14.4196,...,14.0383,15.7610,14.3245,14.9177,13.8960,15.8531,15.3748,13.9527,15.4628,15.1374
9,A4GNT,Gene_AceView,51146,NM_016161.2,3,-,137842560,137851229,10.0039,10.2651,...,9.3946,0.0000,9.4141,11.0339,0.0000,10.9485,11.2048,9.0917,11.0118,9.8772


## Metadata

In [87]:
# Read and print the first few lines of the file
with open("dataset/GSE49711_series_matrix.txt", 'r') as file:
    for _ in range(66):
        print(file.readline())

!Series_title	"RNA-Seq reveals an unprecedented complexity of the neuroblastoma transcriptome and is suitable for clinical endpoint prediction [RNA-Seq]"

!Series_geo_accession	"GSE49711"

!Series_status	"Public on May 22 2015"

!Series_submission_date	"Aug 09 2013"

!Series_last_update_date	"Feb 08 2017"

!Series_pubmed_id	"25150839"

!Series_pubmed_id	"25150838"

!Series_pubmed_id	"25254650"

!Series_summary	"We generated gene expression profiles from 498 primary neuroblastomas using RNA-Seq and microarrays. We sought to systematically evaluate the capability of RNA deep-sequencing (RNA-Seq)-based classification for clinical endpoint prediction in comparison to microarray-based ones. The neuroblastoma cohort was randomly divided into training and validation sets, and 360 predictive models on six clinical endpoints were generated and evaluated. While prediction performances did not differ considerably between the two technical platforms, the RNA-Seq data processing pipelines, or featu

In [158]:
# Read metadata with more controlled handling
metadata_file = "dataset/GSE49711_series_matrix.txt"

# Adjusted this value based on where the data starts as showen above
skip_lines = 66

# Read the file, skipping initial lines and ensuring correct delimiter
metadata = pd.read_csv(metadata_file, sep='\t', skiprows=skip_lines)

# Display first few rows of metadata
metadata.head(50)

,!Sample_title,SEQC_NB001,SEQC_NB002,SEQC_NB003,SEQC_NB004,SEQC_NB005,SEQC_NB006,SEQC_NB007,SEQC_NB008,SEQC_NB009,...,SEQC_NB489,SEQC_NB490,SEQC_NB491,SEQC_NB492,SEQC_NB493,SEQC_NB494,SEQC_NB495,SEQC_NB496,SEQC_NB497,SEQC_NB498
0,!Sample_geo_accession,GSM1205736,GSM1205737,GSM1205738,GSM1205739,GSM1205740,GSM1205741,GSM1205742,GSM1205743,GSM1205744,...,GSM1206224,GSM1206225,GSM1206226,GSM1206227,GSM1206228,GSM1206229,GSM1206230,GSM1206231,GSM1206232,GSM1206233
1,!Sample_status,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,...,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015
2,!Sample_submission_date,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,...,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013
3,!Sample_last_update_date,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,...,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015
4,!Sample_type,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,...,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA
5,!Sample_channel_count,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,!Sample_source_name_ch1,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,...,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma
7,!Sample_organism_ch1,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,...,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens
8,!Sample_characteristics_ch1,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,...,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma
9,!Sample_characteristics_ch1,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,...,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2


# Calculating Gene Signature Scores

### Prepare Metadata

In [159]:
# Transpose the dataframe to get samples as rows
metadata = metadata.T

# Set the first row as the header
metadata.columns = metadata.iloc[0]
metadata = metadata[1:]

# Reset the index
metadata.reset_index(inplace=True)
metadata.columns.name = None
# Rename 'index' column to 'sample_id' for clarity
metadata = metadata.rename(columns={'index': '!Sample_title'})

# Display the first few rows to verify
metadata.head()

,!Sample_title,!Sample_geo_accession,!Sample_status,!Sample_submission_date,!Sample_last_update_date,!Sample_type,!Sample_channel_count,!Sample_source_name_ch1,!Sample_organism_ch1,!Sample_characteristics_ch1,...,!Sample_data_row_count,!Sample_instrument_model,!Sample_library_selection,!Sample_library_source,!Sample_library_strategy,!Sample_relation,!Sample_supplementary_file_1,!series_matrix_table_begin,ID_REF,!series_matrix_table_end
0,SEQC_NB001,GSM1205736,Public on May 22 2015,Aug 09 2013,May 22 2015,SRA,1,neuroblastoma,Homo sapiens,tissue: neuroblastoma,...,0,Illumina HiSeq 2000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE,NaN,GSM1205736,NaN
1,SEQC_NB002,GSM1205737,Public on May 22 2015,Aug 09 2013,May 22 2015,SRA,1,neuroblastoma,Homo sapiens,tissue: neuroblastoma,...,0,Illumina HiSeq 2000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE,NaN,GSM1205737,NaN
2,SEQC_NB003,GSM1205738,Public on May 22 2015,Aug 09 2013,May 22 2015,SRA,1,neuroblastoma,Homo sapiens,tissue: neuroblastoma,...,0,Illumina HiSeq 2000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE,NaN,GSM1205738,NaN
3,SEQC_NB004,GSM1205739,Public on May 22 2015,Aug 09 2013,May 22 2015,SRA,1,neuroblastoma,Homo sapiens,tissue: neuroblastoma,...,0,Illumina HiSeq 2000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE,NaN,GSM1205739,NaN
4,SEQC_NB005,GSM1205740,Public on May 22 2015,Aug 09 2013,May 22 2015,SRA,1,neuroblastoma,Homo sapiens,tissue: neuroblastoma,...,0,Illumina HiSeq 2000,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,NONE,NaN,GSM1205740,NaN


### TGF Beta

In [160]:
# Read the gene list from the file
with open("dataset/HALLMARK_TGF_BETA_SIGNALING", 'r') as file:
    gene_list = file.read().splitlines()

# Remove the first two lines (title and URL)
tgfb_gene_list = gene_list[2:]

# Print the gene list to verify
print(tgfb_gene_list)

['ACVR1', 'APC', 'ARID4B', 'BCAR3', 'BMP2', 'BMPR1A', 'BMPR2', 'CDH1', 'CDK9', 'CDKN1C', 'CTNNB1', 'ENG', 'FKBP1A', 'FNTA', 'FURIN', 'HDAC1', 'HIPK2', 'ID1', 'ID2', 'ID3', 'IFNGR2', 'JUNB', 'KLF10', 'LEFTY2', 'LTBP2', 'MAP3K7', 'NCOR2', 'NOG', 'PMEPA1', 'PPM1A', 'PPP1CA', 'PPP1R15A', 'RAB31', 'RHOA', 'SERPINE1', 'SKI', 'SKIL', 'SLC20A1', 'SMAD1', 'SMAD3', 'SMAD6', 'SMAD7', 'SMURF1', 'SMURF2', 'SPTBN1', 'TGFB1', 'TGFBR1', 'TGIF1', 'THBS1', 'TJP1', 'TRIM33', 'UBE2D3', 'WWTR1', 'XIAP']


In [161]:
# List of Tgfb pathway genes
tgfb_genes = tgfb_gene_list

# Calculate Tgfb signature scores for each sample
tgfb_scores = gene_data[gene_data['Gene'].isin(tgfb_genes)].iloc[:, 8:].mean(axis=0)
tgfb_scores.name = 'tgfb_score'

# Combine tgfb scores with metadata
metadata['tgfb_score'] = metadata['!Sample_title'].map(tgfb_scores)

# Drop samples with missing tgfb scores
metadata.dropna(subset=['tgfb_score'], inplace=True)

In [162]:
tgfb_scores.head()

SEQC_NB001    18.269337
SEQC_NB002    17.887488
SEQC_NB003    17.871847
SEQC_NB004    18.281798
SEQC_NB005    18.154788
Name: tgfb_score, dtype: float64

### IL2 STAT5

In [163]:
# Read the gene list from the file
with open("dataset/HALLMARK_IL2_STAT5_SIGNALING", 'r') as file:
    gene_list = file.read().splitlines()

# Remove the first two lines (title and URL)
il2_stat5_gene_list = gene_list[2:]

# Print the gene list to verify
print(il2_stat5_gene_list)

['ABCB1', 'ADAM19', 'AGER', 'AHCY', 'AHNAK', 'AHR', 'ALCAM', 'AMACR', 'ANXA4', 'APLP1', 'ARL4A', 'BATF', 'BATF3', 'BCL2', 'BCL2L1', 'BHLHE40', 'BMP2', 'BMPR2', 'CA2', 'CAPG', 'CAPN3', 'CASP3', 'CCND2', 'CCND3', 'CCNE1', 'CCR4', 'CD44', 'CD48', 'CD79B', 'CD81', 'CD83', 'CD86', 'CDC42SE2', 'CDC6', 'CDCP1', 'CDKN1C', 'CISH', 'CKAP4', 'COCH', 'COL6A1', 'CSF1', 'CSF2', 'CST7', 'CTLA4', 'CTSZ', 'CXCL10', 'CYFIP1', 'DCPS', 'DENND5A', 'DHRS3', 'DRC1', 'ECM1', 'EEF1AKMT1', 'EMP1', 'ENO3', 'ENPP1', 'EOMES', 'ETFBKMT', 'ETV4', 'F2RL2', 'FAH', 'FGL2', 'FLT3LG', 'FURIN', 'GABARAPL1', 'GADD45B', 'GALM', 'GATA1', 'GBP4', 'GLIPR2', 'GPR65', 'GPR83', 'GPX4', 'GSTO1', 'GUCY1B1', 'HIPK2', 'HK2', 'HOPX', 'HUWE1', 'HYCC2', 'ICOS', 'IFITM3', 'IFNGR1', 'IGF1R', 'IGF2R', 'IKZF2', 'IKZF4', 'IL10', 'IL10RA', 'IL13', 'IL18R1', 'IL1R2', 'IL1RL1', 'IL2RA', 'IL2RB', 'IL3RA', 'IL4R', 'IRF4', 'IRF6', 'IRF8', 'ITGA6', 'ITGAE', 'ITGAV', 'ITIH5', 'KLF6', 'LCLAT1', 'LIF', 'LRIG1', 'LRRC8C', 'LTB', 'MAFF', 'MAP3K8', 'MAP6

In [164]:
# List of il2 stat5 signaling pathway genes
il2_stat5_genes = il2_stat5_gene_list

# Calculate Tgfb signature scores for each sample
il2_stat5_scores = gene_data[gene_data['Gene'].isin(il2_stat5_genes)].iloc[:, 8:].mean(axis=0)
il2_stat5_scores.name = 'il2_stat5_score'

# Combine tgfb scores with metadata
metadata['il2_stat5_score'] = metadata['!Sample_title'].map(il2_stat5_scores)

# Drop samples with missing tgfb scores
metadata.dropna(subset=['il2_stat5_score'], inplace=True)

In [165]:
il2_stat5_scores.head()

SEQC_NB001    17.244859
SEQC_NB002    16.696371
SEQC_NB003    16.120460
SEQC_NB004    17.053701
SEQC_NB005    16.878057
Name: il2_stat5_score, dtype: float64

### IL6 JAK STAT3

In [166]:
# Read the gene list from the file
with open("dataset/HALLMARK_IL6_JAK_STAT3_SIGNALING", 'r') as file:
    gene_list = file.read().splitlines()

# Remove the first two lines (title and URL)
il6_jak_stat3_gene_list = gene_list[2:]

# Print the gene list to verify
print(il6_jak_stat3_gene_list)

['A2M', 'ACVR1B', 'ACVRL1', 'BAK1', 'CBL', 'CCL7', 'CCR1', 'CD14', 'CD36', 'CD38', 'CD44', 'CD9', 'CNTFR', 'CRLF2', 'CSF1', 'CSF2', 'CSF2RA', 'CSF2RB', 'CSF3R', 'CXCL1', 'CXCL10', 'CXCL11', 'CXCL13', 'CXCL3', 'CXCL9', 'DNTT', 'EBI3', 'FAS', 'GRB2', 'HAX1', 'HMOX1', 'IFNAR1', 'IFNGR1', 'IFNGR2', 'IL10RB', 'IL12RB1', 'IL13RA1', 'IL15RA', 'IL17RA', 'IL17RB', 'IL18R1', 'IL1B', 'IL1R1', 'IL1R2', 'IL2RA', 'IL2RG', 'IL3RA', 'IL4R', 'IL6', 'IL6ST', 'IL7', 'IL9R', 'INHBE', 'IRF1', 'IRF9', 'ITGA4', 'ITGB3', 'JUN', 'LEPR', 'LTB', 'LTBR', 'MAP3K8', 'MYD88', 'OSMR', 'PDGFC', 'PF4', 'PIK3R5', 'PIM1', 'PLA2G2A', 'PTPN1', 'PTPN11', 'PTPN2', 'REG1A', 'SOCS1', 'SOCS3', 'STAM2', 'STAT1', 'STAT2', 'STAT3', 'TGFB1', 'TLR2', 'TNF', 'TNFRSF12A', 'TNFRSF1A', 'TNFRSF1B', 'TNFRSF21', 'TYK2']


In [167]:
# List of il6 jak stat3 signaling pathway genes
il6_jak_stat3_genes = il6_jak_stat3_gene_list

# Calculate Tgfb signature scores for each sample
il6_jak_stat3_scores = gene_data[gene_data['Gene'].isin(il6_jak_stat3_genes)].iloc[:, 8:].mean(axis=0)
il6_jak_stat3_scores.name = 'il6_jak_stat3_score'

# Combine tgfb scores with metadata
metadata['il6_jak_stat3_score'] = metadata['!Sample_title'].map(il6_jak_stat3_scores)

# Drop samples with missing tgfb scores
metadata.dropna(subset=['il6_jak_stat3_score'], inplace=True)

In [168]:
il6_jak_stat3_scores.head()

SEQC_NB001    16.482722
SEQC_NB002    15.697078
SEQC_NB003    14.869950
SEQC_NB004    15.792001
SEQC_NB005    15.197942
Name: il6_jak_stat3_score, dtype: float64

### Inflammatory Response

In [169]:
# Read the gene list from the file
with open("dataset/HALLMARK_INFLAMMATORY_RESPONSE", 'r') as file:
    gene_list = file.read().splitlines()

# Remove the first two lines (title and URL)
inflammatory_gene_list = gene_list[2:]

# Print the gene list to verify
print(inflammatory_gene_list)

['ABCA1', 'ABI1', 'ACVR1B', 'ACVR2A', 'ADGRE1', 'ADM', 'ADORA2B', 'ADRM1', 'AHR', 'APLNR', 'AQP9', 'ATP2A2', 'ATP2B1', 'ATP2C1', 'AXL', 'BDKRB1', 'BEST1', 'BST2', 'BTG2', 'C3AR1', 'C5AR1', 'CALCRL', 'CCL17', 'CCL2', 'CCL20', 'CCL22', 'CCL24', 'CCL5', 'CCL7', 'CCR7', 'CCRL2', 'CD14', 'CD40', 'CD48', 'CD55', 'CD69', 'CD70', 'CD82', 'CDKN1A', 'CHST2', 'CLEC5A', 'CMKLR1', 'CSF1', 'CSF3', 'CSF3R', 'CX3CL1', 'CXCL10', 'CXCL11', 'CXCL6', 'CXCL8', 'CXCL9', 'CXCR6', 'CYBB', 'DCBLD2', 'EBI3', 'EDN1', 'EIF2AK2', 'EMP3', 'EREG', 'F3', 'FFAR2', 'FPR1', 'FZD5', 'GABBR1', 'GCH1', 'GNA15', 'GNAI3', 'GP1BA', 'GPC3', 'GPR132', 'GPR183', 'HAS2', 'HBEGF', 'HIF1A', 'HPN', 'HRH1', 'ICAM1', 'ICAM4', 'ICOSLG', 'IFITM1', 'IFNAR1', 'IFNGR2', 'IL10', 'IL10RA', 'IL12B', 'IL15', 'IL15RA', 'IL18', 'IL18R1', 'IL18RAP', 'IL1A', 'IL1B', 'IL1R1', 'IL2RB', 'IL4R', 'IL6', 'IL7R', 'INHBA', 'IRAK2', 'IRF1', 'IRF7', 'ITGA5', 'ITGB3', 'ITGB8', 'KCNA3', 'KCNJ2', 'KCNMB2', 'KIF1B', 'KLF6', 'LAMP3', 'LCK', 'LCP2', 'LDLR', 'LIF'

In [170]:
# List of inflamatory response pathway genes
inflammatory_genes = inflammatory_gene_list

# Calculate Tgfb signature scores for each sample
inflammatory_scores = gene_data[gene_data['Gene'].isin(inflammatory_genes)].iloc[:, 8:].mean(axis=0)
inflammatory_scores.name = 'inflammatory_score'

# Combine tgfb scores with metadata
metadata['inflammatory_score'] = metadata['!Sample_title'].map(inflammatory_scores)

# Drop samples with missing tgfb scores
metadata.dropna(subset=['inflammatory_score'], inplace=True)

In [171]:
inflammatory_scores.head()

SEQC_NB001    15.634172
SEQC_NB002    15.270535
SEQC_NB003    14.382875
SEQC_NB004    15.186789
SEQC_NB005    15.045142
Name: inflammatory_score, dtype: float64

### Interferon Alpha

In [172]:
# Read the gene list from the file
with open("dataset/HALLMARK_INTERFERON_ALPHA_RESPONSE", 'r') as file:
    gene_list = file.read().splitlines()

# Remove the first two lines (title and URL)
ifna_gene_list = gene_list[2:]

# Print the gene list to verify
print(ifna_gene_list)

['ADAR', 'B2M', 'BATF2', 'BST2', 'C1S', 'CASP1', 'CASP8', 'CCRL2', 'CD47', 'CD74', 'CMPK2', 'CMTR1', 'CNP', 'CSF1', 'CXCL10', 'CXCL11', 'DDX60', 'DHX58', 'EIF2AK2', 'ELF1', 'EPSTI1', 'GBP2', 'GBP4', 'GMPR', 'HELZ2', 'HERC6', 'HLA-C', 'IFI27', 'IFI30', 'IFI35', 'IFI44', 'IFI44L', 'IFIH1', 'IFIT2', 'IFIT3', 'IFITM1', 'IFITM2', 'IFITM3', 'IL15', 'IL4R', 'IL7', 'IRF1', 'IRF2', 'IRF7', 'IRF9', 'ISG15', 'ISG20', 'LAMP3', 'LAP3', 'LGALS3BP', 'LPAR6', 'LY6E', 'MOV10', 'MVB12A', 'MX1', 'NCOA7', 'NMI', 'NUB1', 'OAS1', 'OASL', 'OGFR', 'PARP12', 'PARP14', 'PARP9', 'PLSCR1', 'PNPT1', 'PROCR', 'PSMA3', 'PSMB8', 'PSMB9', 'PSME1', 'PSME2', 'RIPK2', 'RNF31', 'RSAD2', 'RTP4', 'SAMD9', 'SAMD9L', 'SELL', 'SLC25A28', 'SP110', 'STAT2', 'TAP1', 'TDRD7', 'TENT5A', 'TMEM140', 'TRAFD1', 'TRIM14', 'TRIM21', 'TRIM25', 'TRIM26', 'TRIM5', 'TXNIP', 'UBA7', 'UBE2L6', 'USP18', 'WARS1']


In [173]:
# List of ifng response pathway genes
ifna_genes = ifna_gene_list

# Calculate Tgfb signature scores for each sample
ifna_scores = gene_data[gene_data['Gene'].isin(ifna_genes)].iloc[:, 8:].mean(axis=0)
ifna_scores.name = 'ifna_score'

# Combine tgfb scores with metadata
metadata['ifna_score'] = metadata['!Sample_title'].map(ifna_scores)

# Drop samples with missing tgfb scores
metadata.dropna(subset=['ifna_score'], inplace=True)

In [174]:
ifna_scores.head()

SEQC_NB001    18.813415
SEQC_NB002    17.277249
SEQC_NB003    16.387085
SEQC_NB004    17.403907
SEQC_NB005    17.422367
Name: ifna_score, dtype: float64

### Interferon Gamma

In [175]:
# Read the gene list from the file
with open("dataset/HALLMARK_INTERFERON_GAMMA_RESPONSE", 'r') as file:
    gene_list = file.read().splitlines()

# Remove the first two lines (title and URL)
ifng_gene_list = gene_list[2:]

# Print the gene list to verify
print(ifng_gene_list)

['ADAR', 'APOL6', 'ARID5B', 'ARL4A', 'AUTS2', 'B2M', 'BANK1', 'BATF2', 'BPGM', 'BST2', 'BTG1', 'C1R', 'C1S', 'CASP1', 'CASP3', 'CASP4', 'CASP7', 'CASP8', 'CCL2', 'CCL5', 'CCL7', 'CD274', 'CD38', 'CD40', 'CD69', 'CD74', 'CD86', 'CDKN1A', 'CFB', 'CFH', 'CIITA', 'CMKLR1', 'CMPK2', 'CMTR1', 'CSF2RB', 'CXCL10', 'CXCL11', 'CXCL9', 'DDX60', 'DHX58', 'EIF2AK2', 'EIF4E3', 'EPSTI1', 'FAS', 'FCGR1A', 'FGL2', 'FPR1', 'GBP4', 'GBP6', 'GCH1', 'GPR18', 'GZMA', 'HELZ2', 'HERC6', 'HIF1A', 'HLA-A', 'HLA-B', 'HLA-DMA', 'HLA-DQA1', 'HLA-DRB1', 'HLA-G', 'ICAM1', 'IDO1', 'IFI27', 'IFI30', 'IFI35', 'IFI44', 'IFI44L', 'IFIH1', 'IFIT1', 'IFIT2', 'IFIT3', 'IFITM2', 'IFITM3', 'IFNAR2', 'IL10RA', 'IL15', 'IL15RA', 'IL18BP', 'IL2RB', 'IL4R', 'IL6', 'IL7', 'IRF1', 'IRF2', 'IRF4', 'IRF5', 'IRF7', 'IRF8', 'IRF9', 'ISG15', 'ISG20', 'ISOC1', 'ITGB7', 'JAK2', 'KLRK1', 'LAP3', 'LATS2', 'LCP2', 'LGALS3BP', 'LY6E', 'LYSMD2', 'MARCHF1', 'METTL7B', 'MT2A', 'MTHFD2', 'MVP', 'MX1', 'MX2', 'MYD88', 'NAMPT', 'NCOA3', 'NFKB1', 'N

In [176]:
# List of ifng response pathway genes
ifng_genes = ifng_gene_list

# Calculate Tgfb signature scores for each sample
ifng_scores = gene_data[gene_data['Gene'].isin(ifng_genes)].iloc[:, 8:].mean(axis=0)
ifng_scores.name = 'ifng_score'

# Combine tgfb scores with metadata
metadata['ifng_score'] = metadata['!Sample_title'].map(ifng_scores)

# Drop samples with missing tgfb scores
metadata.dropna(subset=['ifng_score'], inplace=True)

In [177]:
ifng_scores.head()

SEQC_NB001    18.199822
SEQC_NB002    17.047478
SEQC_NB003    16.125509
SEQC_NB004    17.065319
SEQC_NB005    16.811052
Name: ifng_score, dtype: float64

### Final Metadata

In [178]:
# Transpose metadata back to original form with samples as columns
metadata = metadata.T

# Set the first row as the header
metadata.columns = metadata.iloc[0]
metadata = metadata[1:]

# Reset the index
metadata.reset_index(inplace=True)
metadata.columns.name = None
# Rename 'index' column to 'sample_id' for clarity
metadata = metadata.rename(columns={'index': '!Sample_title'})

,!Sample_title,SEQC_NB001,SEQC_NB002,SEQC_NB003,SEQC_NB004,SEQC_NB005,SEQC_NB006,SEQC_NB007,SEQC_NB008,SEQC_NB009,...,SEQC_NB489,SEQC_NB490,SEQC_NB491,SEQC_NB492,SEQC_NB493,SEQC_NB494,SEQC_NB495,SEQC_NB496,SEQC_NB497,SEQC_NB498
0,!Sample_geo_accession,GSM1205736,GSM1205737,GSM1205738,GSM1205739,GSM1205740,GSM1205741,GSM1205742,GSM1205743,GSM1205744,...,GSM1206224,GSM1206225,GSM1206226,GSM1206227,GSM1206228,GSM1206229,GSM1206230,GSM1206231,GSM1206232,GSM1206233
1,!Sample_status,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,...,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015
2,!Sample_submission_date,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,...,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013
3,!Sample_last_update_date,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,...,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015
4,!Sample_type,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,...,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA


In [179]:
# Display the data to verify
metadata

,!Sample_title,SEQC_NB001,SEQC_NB002,SEQC_NB003,SEQC_NB004,SEQC_NB005,SEQC_NB006,SEQC_NB007,SEQC_NB008,SEQC_NB009,...,SEQC_NB489,SEQC_NB490,SEQC_NB491,SEQC_NB492,SEQC_NB493,SEQC_NB494,SEQC_NB495,SEQC_NB496,SEQC_NB497,SEQC_NB498
0,!Sample_geo_accession,GSM1205736,GSM1205737,GSM1205738,GSM1205739,GSM1205740,GSM1205741,GSM1205742,GSM1205743,GSM1205744,...,GSM1206224,GSM1206225,GSM1206226,GSM1206227,GSM1206228,GSM1206229,GSM1206230,GSM1206231,GSM1206232,GSM1206233
1,!Sample_status,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,...,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015,Public on May 22 2015
2,!Sample_submission_date,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,...,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013,Aug 09 2013
3,!Sample_last_update_date,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,...,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015,May 22 2015
4,!Sample_type,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,...,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA,SRA
5,!Sample_channel_count,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,!Sample_source_name_ch1,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,...,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma,neuroblastoma
7,!Sample_organism_ch1,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,...,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens,Homo sapiens
8,!Sample_characteristics_ch1,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,...,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma,tissue: neuroblastoma
9,!Sample_characteristics_ch1,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,...,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2,dataset: 1,dataset: 2


## Perform Survival Analysis
- **Kaplan-Meier Plot**: The Kaplan-Meier plot will show the survival probability over time for the top 25% and bottom 25% of Tgfb scores.
- **Log-Rank Test**: Perform a log-rank test to determine if the difference in survival between the two groups is statistically significant.

In [ ]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Define event and duration columns
metadata['event'] = metadata['death from disease'].apply(lambda x: 1 if x == 'yes' else 0)
metadata['time'] = metadata['time_to_event_column']  # Replace with actual column name for time to event

# Stratify samples into top 25% and bottom 25%
top_25 = metadata[metadata['tgfb_score'] >= metadata['tgfb_score'].quantile(0.75)]
bottom_25 = metadata[metadata['tgfb_score'] <= metadata['tgfb_score'].quantile(0.25)]

# Kaplan-Meier survival analysis
kmf = KaplanMeierFitter()

plt.figure(figsize=(10, 6))

# Top 25% Tgfb score
kmf.fit(top_25['time'], top_25['event'], label='Top 25% Tgfb Score')
kmf.plot_survival_function()

# Bottom 25% Tgfb score
kmf.fit(bottom_25['time'], bottom_25['event'], label='Bottom 25% Tgfb Score')
kmf.plot_survival_function()

plt.title('Survival Analysis: Top 25% vs Bottom 25% Tgfb Score')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.show()